<a href="https://colab.research.google.com/github/vmartinezarias/Curso_Ecologia_Paisaje_y-Ecoacustica/blob/main/4_Fragmentacion_metricas_paisaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En este script se toma un ráster de coberturas obtenido de Dynamic world, y se emplea el paquete pylandstats para generar las métricas de fragmentación.

Información sobre las métricas pueden ser encontradas en: https://fragstats.org/index.php/fragstats-metrics/patch-based-metrics

Información del paquete pylandstats: https://pylandstats.readthedocs.io/en/latest/index.html

Como resultado, se obtiene un archivo xlsx con las métricas calculadas a nivel de parche, clase y paisaje.


# 1. Instalar los paquetes necesarios en Colab

In [ ]:
!pip install rasterio
!pip install matplotlib
!pip install pylandstats
!pip install pandas
!pip install openpyxl

# 2. Importar bibliotecas necesarias

In [2]:
import rasterio
import matplotlib.pyplot as plt
import pylandstats as pls
import pandas as pd
from google.colab import drive

# 3. Montar Google Drive para acceder al archivo de coberturas .tif

In [ ]:
drive.mount('/content/drive')

# 4. Definir la ruta del archivo .tif en Google Drive


In [4]:
input_filepath = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica/Coberturas_DW.tif'  # Ajusta esta ruta según la ubicación real en Drive


# 5. Cargar el archivo usando pylandstats


In [6]:
ls = pls.Landscape(input_filepath, nodata=-9999)


# 6. Leer y graficar el archivo .tif con rasterio


In [ ]:
with rasterio.open(input_filepath) as src:
    image = src.read(1)  # Lee la primera banda del archivo .tif
plt.figure(figsize=(10, 10))
plt.imshow(image, cmap='viridis')  # Puedes cambiar el colormap según tus preferencias
plt.colorbar(label="Valores de pixel")
plt.title("Visualización del archivo .tif")
plt.xlabel("Coordenadas X")
plt.ylabel("Coordenadas Y")
plt.show()

# 7. Diccionario para mapear los valores de clase a sus nombres de coberturas
Se debe adaptar dependiendo de si se usa otro tipo de insumo

In [8]:
class_names = {
    0: "agua",
    1: "árboles",
    2: "pastos",
    3: "vegetación_inundada",
    4: "cultivos",
    5: "matorral_y_arbustos",
    6: "construido",
    7: "desnudo",
    8: "nieve_y_hielo"
}

# 8. Calcular métricas de parches y limpiar NaN


In [ ]:
patch_metrics_df = ls.compute_patch_metrics_df()
patch_metrics_df_clean = patch_metrics_df.dropna()
patch_metrics_df_clean['class_name'] = patch_metrics_df_clean['class_val'].map(class_names)
print(patch_metrics_df_clean)

# 9. Verificar clases con menos de x número de parches

Se puede adaptar esta parte para identificar clases con x número de parches


In [ ]:

if patch_metrics_df_clean.empty:
    print("Error: El DataFrame 'patch_metrics_df_clean' está vacío.")
else:

    print("Columnas de patch_metrics_df_clean:", list(patch_metrics_df_clean.columns))


    if 'class_name' in patch_metrics_df_clean.columns:
        print("La columna 'class_name' fue encontrada en el DataFrame.")


        classes_with_few_patches = patch_metrics_df_clean['class_name'].value_counts()


        print("Conteo de ocurrencias por clase:")
        print(classes_with_few_patches)


        few_patch_classes = classes_with_few_patches[classes_with_few_patches < 8]  # Cambiar el límite aquí si es necesario

        print("Clases con menos de cinco parches:")
        print(few_patch_classes)
    else:
        print("Error: La columna 'class_name' no existe en el DataFrame 'patch_metrics_df_clean'.")




# 10. Calcular métricas de clase y limpiar NaN

In [ ]:
class_metrics_df = ls.compute_class_metrics_df()
class_metrics_df_clean = class_metrics_df.dropna().reset_index()
class_metrics_df_clean['class_name'] = class_metrics_df_clean['class_val'].map(class_names)
print(class_metrics_df_clean)

# 11. Calcular métricas de paisaje


In [ ]:
landscape_metrics_df = ls.compute_landscape_metrics_df()
landscape_metrics_df
print(landscape_metrics_df)

# 12. Exportar los resultados a un archivo Excel en Google Drive


In [ ]:
import os
import pandas as pd

# Ruta de la carpeta principal y la subcarpeta "Resultados"
main_folder_path = '/content/drive/MyDrive/Curso_Ecologia_Paisaje_Ecoacustica'
results_folder_path = os.path.join(main_folder_path, "Resultados")
if not os.path.exists(results_folder_path):
    os.makedirs(results_folder_path)
    print(f"Subcarpeta 'Resultados' creada en {results_folder_path}")
else:
    print("La subcarpeta 'Resultados' ya existe.")

excel_path = os.path.join(results_folder_path, 'landscape_metrics.xlsx')

# Guardar los DataFrames en el archivo Excel con diferentes hojas
with pd.ExcelWriter(excel_path) as writer:
    patch_metrics_df_clean.to_excel(writer, sheet_name='Métricas de parche', index=False)
    class_metrics_df_clean.to_excel(writer, sheet_name='Métricas de clase', index=False)
    landscape_metrics_df.to_excel(writer, sheet_name='Métricas de Paisaje', index=False)

print(f"Archivo Excel guardado en: {excel_path}")
